First, we remove NaN's and \\N values from the dataset, but only where it matters, i.e. at IATA (airport code), latitude, longitude, country and city

In [181]:
import pandas as pd

cols = ["id", "city", "country", "IATA", "ICAO", "latitude", "longitude", "altitude", "timezone", "DST", "Tz database time zone", "type", "source"]
cols_to_remove = ["ICAO", "altitude", "timezone", "DST", "Tz database time zone", "type", "source"]
airports = pd.read_csv('data/airports.csv', index_col=0, names=cols, header=None)
airports = airports.drop(cols_to_remove, axis=1)
print(len(airports))

subset = ["IATA", "latitude", "longitude", "country", "city"]
airports = airports.dropna(subset=subset)
for col in subset:
    airports = airports[airports[col] != "\\N"]

7698


Next, we make a dictionary mapping from country to continent, such that we can add continent to the above dataframe.

In [182]:
continents = pd.read_csv('data/continents.csv')
## make a dictionary of country to continent
country_to_continent = {}
for index, row in continents.iterrows():
    country = row['Entity']
    country_to_continent[row['Entity']] = row['Continent']
    
## show the first 5 entries of the dictionary
dict(list(country_to_continent.items())[0:5])

{'Abkhazia': 'Asia',
 'Afghanistan': 'Asia',
 'Akrotiri and Dhekelia': 'Asia',
 'Aland Islands': 'Europe',
 'Albania': 'Europe'}

Similarily, we make a dictionary that maps from country to BNP, such that this can be added to the dataframe aswell. 
We only consider BNP from 2015, since this also around the time that the reviews are  gathered. 

In [183]:
BNP_per_capita = pd.read_csv('data/gdp-per-capita-worldbank.csv')
BNP_per_capita_2015 = BNP_per_capita[BNP_per_capita["Year"] == 2015]
## make a dictionary of country to BNP per capita
country_to_BNP_per_capita = {}
for index, row in BNP_per_capita_2015.iterrows():
    country = row['Entity']
    country_to_BNP_per_capita[country] = row['GDP per capita, PPP (constant 2017 international $)']
## see first 5 entries of the dictionary
dict(list(country_to_BNP_per_capita.items())[0:5])

{'Afghanistan': 2108.714,
 'Albania': 11880.598,
 'Algeria': 11751.634,
 'Angola': 7967.104,
 'Antigua and Barbuda': 19345.018}

The data of continents/BNP comes from Our World In Data (owid), and it doesn't match with the country names from the airport dataset. Therefore, we manually have to fix these names. 

In [184]:
countries_from_airports = airports['country'].unique()
countries_from_continents = continents['Entity'].unique()
countries_from_BNP = BNP_per_capita_2015['Entity'].unique()
## make a set of unique countries from continents and BNP
unique_countries = set(countries_from_continents).union(set(countries_from_BNP))
unique_countries = list(unique_countries)

## find the countries that are in airports but not in continents
print("These names should be fixed:")
for country in countries_from_airports:
    if country not in unique_countries:
        print(country)
        
def fix_country_name(name : str) -> str:
    """
    Function for matching up the country names in the airports dataset with the country names in the continents dataset.
    """
    
    if name[:5] == "Congo":
        return "Congo"
    
    if name == "Czech Republic":
        return "Czechia"
    
    if name == "Macedonia":
        return "North Macedonia"
    
    if name == "Swaziland":
        return "Eswatini"
    
    if name == "Micronesia":
        return "Micronesia (country)"
    
    if name == "Burma":
        return "Myanmar"
    
    if name in ["Midway Islands", "US Virgin Islands", "Virgin Islands", "Johnston Atoll", "Wake Island"]:
        return "United States"
    
    if name == "Macau":
        return "China"
    
    if name == "Cocos (Keeling) Islands":
        return "Australia"
    
    if name == "Russian Federation":
        return "Russia"
    
    return name

These names should be fixed:
Congo (Brazzaville)
Congo (Kinshasa)
Swaziland
Czech Republic
Macedonia
Midway Islands
Micronesia
Virgin Islands
Macau
Burma
Johnston Atoll
Cocos (Keeling) Islands
Wake Island


In [189]:
## make a new column in the airports dataset with the continent
airports['continent'] = airports['country'].apply(fix_country_name).map(country_to_continent)
## same for the BNP per capita
airports['BNP per capita'] = airports['country'].apply(fix_country_name).map(country_to_BNP_per_capita)
## if the BNP is a nan, fill it with the mean
airports['BNP per capita'] = airports['BNP per capita'].fillna(airports['BNP per capita'].mean())
## save the dataset
airports.to_csv('data/airports_cleaned.csv')

In [177]:
from fuzzywuzzy import process
from tqdm import tqdm
from functools import cache
tqdm.pandas()

def format_name(name):
    name = name.replace("-", " ")
    ## make first letter in each word uppercase
    name = " ".join([word.capitalize() for word in name.split()])
    return name

@cache
def match_name(name, candidates):
    name = format_name(name)
    if name in candidates:
        return name
    
    ## get the best match
    match, score = process.extractOne(name, candidates)
    if score > 90:
        return match
    
    return None

airportreviews = pd.read_csv("data/airportreviews.csv")
cols_to_keep = ["airport_name", "recommended", "content"]
airportreviews = airportreviews[cols_to_keep]
airportreviews = airportreviews.dropna(subset=cols_to_keep)

candidates = airports['id'].unique()
candidates = tuple(candidates)
airportreviews["matched_name"] = airportreviews["airport_name"].progress_apply(lambda x: match_name(x, candidates))
airportreviews = airportreviews.dropna(subset=["matched_name"])

100%|██████████| 17721/17721 [02:59<00:00, 98.64it/s]  


In [186]:
## for each airport in airports dataframe, insert the average recommendation score from airportsreviews
airportreviews["recommended"] = airportreviews["recommended"].astype(int)
average_recommendation = airportreviews.groupby("matched_name")["recommended"].mean()
reviews_as_list = airportreviews.groupby("matched_name")["content"].apply(list).reset_index()

airports = pd.merge(airports, reviews_as_list, how="left", left_on="id", right_on="matched_name")
airports = pd.merge(airports, average_recommendation, how="left", left_on="id", right_on="matched_name")

## change column name to "average_recommendation"
airports = airports.rename(columns={"recommended": "average_recommendation"})

Now, using the dictionaries, we can fill in BNP and continents of the airports. 

In [192]:
## make a dictionary of IATA codes
## IATA code is a three-letter code designating many airports around the world
## for each airport, safe relevant information in a dictionary
from tqdm import tqdm
import numpy as np

IATA = {}
for index, row in airports.iterrows():
    country = row['country']
    
    IATA[row['IATA']] = {
        'city': row['city'],                                                ## city
        'country': country,                                                 ## country
        'latitude': row['latitude'],                                        ## latitude
        'longitude': row['longitude'],                                      ## longitude
        'name': row['id'],                                                  ## name        
        'continent': country_to_continent.get(fix_country_name(country), 'unknown'),    ## continent
        'BNP_per_capita': country_to_BNP_per_capita.get(fix_country_name(country), 'unknown'), ## BNP per capita
        'average_recommendation' : row['average_recommendation'],           ## average recommendation
    }
    
## save the dictionary to a file
import json
with open('data/IATA.json', 'w') as f:
    json.dump(IATA, f)

## show the first 5 entries of the dictionary
for i, (k, v) in enumerate(IATA.items()):
    print(k, v)
    if i == 5:
        break
print("...")
    
## count how many airports are in each continent
continent_count = {}
for k, v in IATA.items():
    continent = v['continent']

    if continent not in continent_count:
        continent_count[continent] = 0
    continent_count[continent] += 1
    
print("Number of airports in each continent:")
print(continent_count)

GKA {'city': 'Goroka', 'country': 'Papua New Guinea', 'latitude': -6.081689834590001, 'longitude': 145.391998291, 'name': 'Goroka Airport', 'continent': 'Oceania', 'BNP_per_capita': 3813.1143, 'average_recommendation': nan}
MAG {'city': 'Madang', 'country': 'Papua New Guinea', 'latitude': -5.20707988739, 'longitude': 145.789001465, 'name': 'Madang Airport', 'continent': 'Oceania', 'BNP_per_capita': 3813.1143, 'average_recommendation': nan}
HGU {'city': 'Mount Hagen', 'country': 'Papua New Guinea', 'latitude': -5.826789855957031, 'longitude': 144.29600524902344, 'name': 'Mount Hagen Kagamuga Airport', 'continent': 'Oceania', 'BNP_per_capita': 3813.1143, 'average_recommendation': nan}
LAE {'city': 'Nadzab', 'country': 'Papua New Guinea', 'latitude': -6.569803, 'longitude': 146.725977, 'name': 'Nadzab Airport', 'continent': 'Oceania', 'BNP_per_capita': 3813.1143, 'average_recommendation': nan}
POM {'city': 'Port Moresby', 'country': 'Papua New Guinea', 'latitude': -9.44338035583496, 'long

In [13]:
## find all unique airport names
airports_in_reviews = reviews['airport_name'].unique()
print("Number of unique airport names in reviews:", len(airports_in_reviews))

Number of unique airport names in reviews: 425


In [14]:
airports_in_dataset = airports["id"].apply(lambda x: x.lower()).unique().tolist()
print("Number of unique airport names in dataset:", len(airports_in_dataset))

Number of unique airport names in dataset: 6007


In [15]:
## airports in reviews but not in dataset
n = 0
for airport in airports_in_reviews:
    if airport.lower() not in airports_in_dataset:
        print(airport)
        n += 1
        
print("Number of airports in reviews but not in dataset:", n)

ærø airport
altenburg nobitz airport
maritsa airport
bantayan airport
jastarnia airport
mangalore airport
relizane airport
berlin schönefeld airport
berlin tegel airport
birmingham shuttlesworth international airport
vumbura airport
embu airport
metropolitano airport
clear airport
cleve airport
colorado springs east airport
chofu airport
tobago crown point airport
cutral co airport
denali airport
djibouti ambouli airport
edmundston airport
fairoaks airport
frankfurt hahn airport
greytown airport
gothenburg landvetter airport
grenoble isère airport
hanko airport
hancock county bar harbor airport
burg airport
princeton airport
john wayne airport orange county airport
kabale airport
tokachi airport
karlshöfen airport
kazarman airport
west kilimanjaro airport
kidston airport
la rochelle île de ré airport
ulongwe airport
lusaka city airport
lyon saint exupéry airport
millington memphis airport
nice côte d'azur airport
oakland troy airport
deesa airport
palmar airport
alphonse airport
paris 

In [16]:
cols = ["airline", "airline id", "source airport", "source airport id", "destination airport", "destination airport id", "codeshare", "stops", "equipment"]
flights = pd.read_csv('data/flights.csv', names=cols, header=None)
print(len(flights))
flights = flights.dropna(subset=['source airport', 'destination airport'])
flights = flights[flights['stops'] == 0]  ## only direct flights
print(len(flights))
flights

67663
67652


,airline,airline id,source airport,source airport id,destination airport,destination airport id,codeshare,stops,equipment
0,2B,410,AER,2965,KZN,2990,NaN,0,CR2
1,2B,410,ASF,2966,KZN,2990,NaN,0,CR2
2,2B,410,ASF,2966,MRV,2962,NaN,0,CR2
3,2B,410,CEK,2968,KZN,2990,NaN,0,CR2
4,2B,410,CEK,2968,OVB,4078,NaN,0,CR2
...,...,...,...,...,...,...,...,...,...
67658,ZL,4178,WYA,6334,ADL,3341,NaN,0,SF3
67659,ZM,19016,DME,4029,FRU,2912,NaN,0,734
67660,ZM,19016,FRU,2912,DME,4029,NaN,0,734
67661,ZM,19016,FRU,2912,OSS,2913,NaN,0,734


In [17]:
## makes a graph of the flights
## the graph is a directed graph since the flights are one way
## nodes are cities, and edges are flights between cities
## the weight of the edge is the number of flights between the cities

import networkx as nx
from tqdm import tqdm
G = nx.DiGraph()

print("Making nodes with attributes..")
for iata, data in tqdm(IATA.items()):
    G.add_node(
        iata, 
        city=data['city'], 
        country=data['country'], 
        latitude=data['latitude'], 
        longitude=data['longitude'], 
        continent=data['continent'],
        name=data['name'],
        
        group=data['continent']
        )

print("Making edges..")
for index, row in tqdm(flights.iterrows(), total=flights.shape[0]):
    source = row['source airport']
    dest = row['destination airport']
    if source in IATA.keys() and dest in IATA.keys():
        if G.has_edge(source, dest):
            G[source][dest]['weight'] += 1
        else:
            G.add_edge(source, dest, weight=1)
            
## remove nodes with no edges
G.remove_nodes_from(list(nx.isolates(G)))

## 
import pickle
with open('data/graphnetwork.gpickle', 'wb') as f:
    pickle.dump(G, f, pickle.HIGHEST_PROTOCOL)

print("Number of nodes: ", G.number_of_nodes())
print("Number of edges: ", G.number_of_edges())

Making nodes with attributes..


100%|██████████| 6033/6033 [00:00<00:00, 117006.23it/s]


Making edges..


100%|██████████| 67652/67652 [00:07<00:00, 9440.51it/s] 

Number of nodes:  3256
Number of edges:  37038
